In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import statsmodels.api as sm


In [2]:

def Fama_MacBeth(portfolio2,factor_model_raw):
    ####此处factor_model为n*t矩阵，n为因子数目，t为时间范围
    ####此处portfolio2为m*t矩阵，m为投资组合个数，t为时间范围
    m=np.size(portfolio2,axis=0)
    t=np.size(portfolio2,axis=1)
    factor_model=factor_model_raw.copy().T
    factor_model_name=list(factor_model.columns)
    factor_model.columns=factor_model_name
    ports_beta=pd.DataFrame(columns=factor_model_name)
    estimates_lambda=pd.DataFrame(columns=factor_model_name)
    R_squares=[]
    regression_result=pd.DataFrame(columns=list(factor_model_name)+['R_squares'],index=['tvalues','estimate','R_squares'])
    for num_port in range(0,m):
        model1=sm.OLS(portfolio2.iloc[num_port,:].astype(float),np.array(factor_model).astype(float))
        port_beta=model1.fit().params
        port_beta.index=factor_model_name
        ports_beta.loc[num_port,:]=port_beta
    for time in range(0,t):
        model2=sm.OLS(portfolio2.iloc[:,time].astype(float),np.array(ports_beta).astype(float))
        estimate_lambda=model2.fit().params
        estimate_lambda.index=ports_beta.columns
        estimates_lambda.loc[time,:]=estimate_lambda
        R_squares.append(model2.fit().rsquared_adj)
    for factor in factor_model_name:
            t = (np.mean(estimates_lambda[factor]))/(np.std(estimates_lambda[factor])/np.sqrt(len(estimates_lambda[factor])))
            mean=np.mean(estimates_lambda[factor])
            regression_result[factor]['tvalues']=t
            regression_result[factor]['estimate']=mean
    regression_result['R_squares']=np.mean(R_squares)
    return regression_result

In [3]:
#导入收益数据和因子数据
portfolio2=pd.read_excel(r'portfolio.xlsx',header=None)
factorm=pd.read_excel(r'gefactor-beta.xlsx',header=None)



In [4]:
data_times='1/1/2006'
data_timee='12/1/2017'
time_seri_total=pd.date_range(start = '1/1/2000',end='12/1/2017',freq = 'MS')
time_seri=pd.date_range(start = data_times,end=data_timee,freq = 'MS')
data_ticks=np.where(time_seri_total==data_times)[0][0]
data_ticke=np.where(time_seri_total==data_timee)[0][0]
copyfactor=factorm.copy()
copyportfolio2=portfolio2.copy()
portfolio2=portfolio2.iloc[:,data_ticks:data_ticke+1]
factorm=pd.DataFrame(np.array(pd.concat([factorm.copy().iloc[:,0:2],factorm.copy().iloc[:,data_ticks+2:data_ticke+3]],axis=1)))


In [5]:
#加表头
factorname=factorm.copy().iloc[:,0]
for j in range(0,len(factorname)):
    id_=np.where(np.array([i=='_' for i in factorname[j]])==1)[0]
    if np.size(id_)!=0:
        factorname[j]=factorname[j][id_[0]+1:id_[-1]]
factordate=factorm[:][1]
factoryear=np.array(factorm.iloc[:,1])

factorm=factorm.iloc[:,2:]
factorm.index=factorname
factorm.columns=time_seri
factorm.head()

,2006-01-01,2006-02-01,2006-03-01,2006-04-01,2006-05-01,2006-06-01,2006-07-01,2006-08-01,2006-09-01,2006-10-01,...,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01
0,,,,,,,,,,,,,,,,,,,,,
ana,-0.113058,-0.0124199,-0.0085909,-0.0976053,-0.226064,-0.0272245,-0.0357569,0.00594472,0.0197881,0.0387073,...,0.0415319,0.0576087,0.0908136,0.019485,0.00164577,0.0106355,0.0175537,0.0666901,0.0543527,0.0404576
rea,-0.0385967,0.0282971,-0.00011907,0.000848556,-0.116703,-0.0160533,-0.0350801,0.00558936,0.0227809,0.0396965,...,0.0371947,0.0530487,0.0905275,0.017412,-0.0149584,0.00962947,0.0189902,0.0742942,0.0504454,0.0397583
gp,-0.0110016,0.0245284,0.0488265,0.050175,-0.0735145,0.000263535,-0.037663,-0.00800099,-0.00924077,0.042346,...,0.0475963,0.0400699,0.064816,0.0287453,-0.0607606,-0.00139744,0.0390617,0.080157,0.00246449,0.0573114
roa,-0.000363097,-0.0259199,0.0450858,0.060168,-0.0582424,-0.0162528,-0.0272808,-0.0093553,-0.0434004,-0.00351388,...,0.062928,0.0582967,0.083127,0.01267,-0.0837261,-0.00198232,0.0456216,0.0564168,-0.0109034,0.0444532
roe,0.0144066,0.00587193,0.0404702,0.0473738,-0.118082,-0.0263266,-0.0477677,0.00883976,-0.0161044,0.0548569,...,0.045349,0.058136,0.0966783,-0.00171572,-0.0663033,-0.0124712,0.014391,0.0746306,0.037001,0.0300732


In [6]:
#实现Fama_Macbeth回归
model_name=['market','size','dbe','bm','roe']
factor_model_raw=factorm.loc[model_name,:]
Fama_result=Fama_MacBeth(portfolio2,factor_model_raw)
print(Fama_result)

              market       size         dbe          bm         roe  R_squares
tvalues      2.25953    -2.6646    -1.86552   -0.230179   -0.256874   0.893941
estimate   0.0167372 -0.0181201 -0.00658173 -0.00109912 -0.00118047   0.893941
R_squares        NaN        NaN         NaN         NaN         NaN   0.893941


In [7]:
Fama_result.to_csv('Fama_result.csv')